# Privacy-Preserving RAG: Clinical Assistant with Tool Calling

This notebook implements a clinical assistant powered by OpenAI's O3 model that can answer patient-specific questions using structured data while preserving privacy through local de-identification.

## System Overview
- **OpenAI O3 Agent**: Acts as orchestrator and QA engine
- **Tool Functions**: Retrieve patient data from various sources
- **Local De-identifier**: Removes PHI before sending to OpenAI
- **Privacy-Safe Responses**: Clinically informed answers without compromising privacy

In [1]:
from fetch_data import generate_note_auto;

In [2]:
# Install essential packages
# Run this cell first to install core dependencies

!pip install openai python-dotenv pandas numpy 
!pip install ipywidgets


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [5]:
# Import necessary libraries
import os
import json
from typing import Dict, List, Any, Optional, Union
from datetime import datetime, date
import pandas as pd
import numpy as np

# OpenAI imports
from openai import OpenAI

# Environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# gated model login with Hugging Face CLI
# Make sure you have the Hugging Face CLI installed and authenticated
!pip install huggingface_hub
!pip install llama-cpp-python


from huggingface_hub import login
hugginfface_token = os.getenv("HUGGING_FACE_API_KEY")
login(hugginfface_token)

from llama_cpp import Llama

# Download the model to 'models/gemma-3-4b-it-qat-q4_0.gguf' as per previous steps
llm = Llama.from_pretrained(
    repo_id="google/gemma-3-4b-it-qat-q4_0-gguf",
    filename="gemma-3-4b-it-q4_0.gguf",  # ✅ correct filename
    n_ctx=2048,
    n_threads=8
)


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


llama_model_load_from_file_impl: using device Metal (Apple M2) - 10916 MiB free
llama_model_loader: loaded meta data with 39 key-value pairs and 444 tensors from /Users/arunjoshi/.cache/huggingface/hub/models--google--gemma-3-4b-it-qat-q4_0-gguf/snapshots/15f73f5eee9c28f53afefef5723e29680c2fc78a/./gemma-3-4b-it-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma3
llama_model_loader: - kv   1:                      gemma3.context_length u32              = 131072
llama_model_loader: - kv   2:                         gemma3.block_count u32              = 34
llama_model_loader: - kv   3:                    gemma3.embedding_length u32              = 2560
llama_model_loader: - kv   4:                 gemma3.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                gemma3.attention.he

In [7]:
# Updated De-identification with Correct Context Window
import re
from typing import List, Tuple

def estimate_tokens(text: str) -> int:
    """Roughly estimates token count, assuming 1 token ≈ 4 characters."""
    return len(text) // 4

def merge_deidentified_chunks(chunks: List[str]) -> str:
    """Merges de-identified text chunks back into a single document."""
    if not chunks:
        return ""
    return '\n\n'.join(chunks)

# --- NEW: Token-based splitter ---
def split_text_by_tokens(text: str, llm_model: Llama, max_tokens: int, overlap_tokens: int) -> List[str]:
    """Splits text into chunks of a specific token size using the model's tokenizer."""
    print(".. using precise token-based splitter ..")
    
    # 1. Tokenize the entire text using the model's tokenizer
    tokens = llm_model.tokenize(text.encode("utf-8", "ignore"))
    
    # 2. Create chunks of tokens
    token_chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        token_chunks.append(tokens[start:end])
        
        # If this is the last chunk, we're done
        if end == len(tokens):
            break
            
        # Move the start pointer back by the overlap amount, ensuring it doesn't get stuck
        next_start = end - overlap_tokens
        start = max(next_start, start + 1)

    # 3. Convert token chunks back into text
    text_chunks = [llm_model.detokenize(chunk).decode("utf-8", "ignore") for chunk in token_chunks]
    return text_chunks

In [8]:
# The system prompt that instructs the model
deidentifier_prompt = """
Remove all HIPAA-protected information and generate a concise clinical summary. Output only the de-identified summary, with no extra conversational text.

Follow these rules strictly:
1.  Remove all Protected Health Information (PHI) including: Names, Addresses/ZIP codes, All elements of dates except for the year, Phone/fax numbers, Email addresses, Social Security numbers, Medical record/account numbers, License numbers, Vehicle/device identifiers, URLs/IP addresses, Biometric identifiers, and Full-face photos.
2.  Preserve all clinically relevant information.
3.  Structure the output using clear headings and bullet points.
4.  Maintain chronological order where applicable (most recent events first).
"""

# --- Model & Chunking Parameters ---
# Set to the model's native context size
MODEL_CONTEXT_WINDOW = 8192
# Reserve tokens for the model's response
MAX_OUTPUT_TOKENS = 2048
# Small buffer for safety
SAFETY_BUFFER_TOKENS = 100

# --- Calculated Values (Do not change these directly) ---
PROMPT_TOKENS = estimate_tokens(deidentifier_prompt)
# Calculate the maximum number of tokens available for user input per chunk
MAX_INPUT_TOKENS = MODEL_CONTEXT_WINDOW - PROMPT_TOKENS - MAX_OUTPUT_TOKENS - SAFETY_BUFFER_TOKENS
# Convert the token limit to an approximate character limit for the splitter
MAX_CHUNK_CHARACTERS = MAX_INPUT_TOKENS * 4
# Character overlap between chunks to maintain context
CHUNK_OVERLAP = 250

In [9]:
# --- Authenticate and Load Model ---
print("⚙️  Initializing model...")

# If you need to log in (usually only required once)
# hf_token = os.getenv("HUGGING_FACE_API_KEY")
# login(token=hf_token)

# Load the model from the Hugging Face Hub
llm = Llama.from_pretrained(
    repo_id="google/gemma-3-4b-it-qat-q4_0-gguf",
    filename="gemma-3-4b-it-q4_0.gguf",
    n_ctx=MODEL_CONTEXT_WINDOW,  # Use the full 8K context
    n_threads=8,                 # Adjust based on your CPU cores
    verbose=False                # Set to True for detailed logs
)

print("✅ Model initialized successfully.")

⚙️  Initializing model...


llama_context: n_ctx_per_seq (8192) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64

✅ Model initialized successfully.


In [ ]:
def deidentify_chunk(chunk: str) -> str:
    """Sends a single chunk to the LLM for de-identification."""
    try:
        response = llm.create_chat_completion(
            messages=[
                {"role": "system", "content": deidentifier_prompt},
                {"role": "user", "content": chunk}
            ],
            max_tokens=MAX_OUTPUT_TOKENS,
            temperature=0.2
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"❌ Error during chunk processing: {e}")
        return f"[DEIDENTIFICATION_ERROR: {str(e)}]"

def deidentify(user_data: str) -> str:
    """Main function to de-identify text, using token-based chunking."""
    print("\n🔍 Starting de-identification process...")
    # We use the model to get an exact token count
    total_tokens = len(llm.tokenize(user_data.encode("utf-8", "ignore")))
    print(f"📏 Input size: {len(user_data)} characters ({total_tokens} tokens)")

    # Check if the text is small enough to process in one go
    if total_tokens <= MAX_INPUT_TOKENS:
        print("✅ Text fits in context window, processing as a single chunk.")
        deidentified_text = deidentify_chunk(user_data)
    else:
        print(f"📊 Text exceeds safe input size of {MAX_INPUT_TOKENS} tokens. Splitting into chunks...")
        
        # --- USE THE NEW TOKEN SPLITTER ---
        chunks = split_text_by_tokens(
            user_data,
            llm_model=llm,
            max_tokens=MAX_INPUT_TOKENS,
            overlap_tokens=CHUNK_OVERLAP
        )
        print(f"🔧 Split into {len(chunks)} chunks.") # This number will now be much smaller and correct

        deidentified_chunks = []
        for i, chunk in enumerate(chunks):
            chunk_token_count = len(llm.tokenize(chunk.encode("utf-8", "ignore")))
            print(f"🔄 Processing chunk {i+1}/{len(chunks)} ({chunk_token_count} tokens)...")
            deidentified_chunk = deidentify_chunk(chunk)
            deidentified_chunks.append(deidentified_chunk)

        print("🔗 Merging de-identified chunks...")
        deidentified_text = merge_deidentified_chunks(deidentified_chunks)

    print(f"✅ De-identification complete. Output size: {len(deidentified_text)} characters.")
    return deidentified_text

In [11]:
print("TEST DEIDENTIFICATION")
patient_notes = generate_note_auto("observations", "e7a5d3dc-3484-a24e-6ef3-0737b403f950",)
deidentified_notes = deidentify(patient_notes)
print("original patient notes:", patient_notes)
print("Deidentified Notes:", deidentified_notes)

TEST DEIDENTIFICATION

🔍 Starting de-identification process...
📏 Input size: 1246 characters (558 tokens)
✅ Text fits in context window, processing as a single chunk.


/Volumes/Nirvana/masters/AI in healthcare/final_project/fetch_data.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_rows['date_only'] = pd.to_datetime(patient_rows['recorded_ts']).dt.date


✅ De-identification complete. Output size: 489 characters.
original patient notes: 
📌 Patient Information (ID: e7a5d3dc-3484-a24e-6ef3-0737b403f950)
- Full Name: Marni Tremblay
- Birthdate: 1976-10-20
- Ssn: 999-36-6337
- Drivers: S99999779
- Passport: X42697334X
- Prefix: Mrs.
- Marital: D
- Race: white
- Ethnicity: nonhispanic
- Gender: F
- Birthplace: Park River  North Dakota  US
- Patient Address: 118 Kiehn Gardens Suite 40
- Patient City: West Fargo
- Patient State: North Dakota
- County: Cass County
- Fips: 38017.0
- Patient Zip: 58078
- Lat: 46.8934021332706
- Lon: -96.8621390602698
- Healthcare Expenses: 157011.5
- Healthcare Coverage: 1055670.27
- Income: 29501

🧪 Observations:
- [2025-04-12 09:00:18.000000]: Diastolic Blood Pressure (Code: 8462-4) = 61.0 mm[Hg] | Type: numeric | Category: vital-signs | Age: 48 | Provider: Otha Roberts (GENERAL PRACTICE) | Org: ESSENTIA HEALTH FARGO - WEST FARGO, ND | Encounter: Follow-up encounter (procedure) | Total Cost: $96.45 (Payer Cover

In [12]:
# OpenAI Configuration
class ClinicalAssistantConfig:
    """Configuration for the Clinical Assistant"""
    
    def __init__(self):
        # OpenAI API configuration
        self.api_key = os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            print("⚠️  Warning: OPENAI_API_KEY not found in environment variables")
            print("💡 Please set your OpenAI API key in a .env file or environment variable")
        
        # Model configuration
        self.model = "gpt-4.1"  # Using GPT-4.1
        self.max_completion_tokens = 4000  # Use max_completion_tokens for newer models
        self.temperature = 0.1  # Low temperature for clinical accuracy
        
        # Tool calling configuration
        self.max_tool_calls = 5
        self.parallel_tool_calls = True

# Initialize configuration
config = ClinicalAssistantConfig()

# Initialize OpenAI client
if config.api_key:
    client = OpenAI(api_key=config.api_key)
    print("✅ OpenAI client initialized successfully!")
    print(f"🤖 Model: {config.model}")
else:
    client = None
    print("❌ OpenAI client not initialized - API key required")

✅ OpenAI client initialized successfully!
🤖 Model: gpt-4.1


In [13]:
# Tool Function Definitions for OpenAI Function Calling
# These will be used by the O3 model to understand available tools

TOOL_DEFINITIONS = [
    {
        "type": "function",
        "function": {
            "name": "get_patient_observations",
            "description": "Retrieve laboratory test results and clinical observations for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_conditions",
            "description": "Retrieve patient diagnosis information, medical conditions, and medical history",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_medications",
            "description": "Retrieve current and past medications for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_careplans",
            "description": "Retrieve care plans and treatment plans for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_procedures",
            "description": "Retrieve medical procedures and interventions performed on a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_imaging_studies",
            "description": "Retrieve imaging studies and radiology reports for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_immunizations",
            "description": "Retrieve vaccination history and immunization records for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_allergies",
            "description": "Retrieve allergy information and adverse reactions for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    }
]

print("✅ Tool function definitions created!")
print(f"🔧 Available tools: {len(TOOL_DEFINITIONS)}")
for tool in TOOL_DEFINITIONS:
    print(f"   - {tool['function']['name']}: {tool['function']['description']}")

✅ Tool function definitions created!
🔧 Available tools: 8
   - get_patient_observations: Retrieve laboratory test results and clinical observations for a patient
   - get_patient_conditions: Retrieve patient diagnosis information, medical conditions, and medical history
   - get_patient_medications: Retrieve current and past medications for a patient
   - get_patient_careplans: Retrieve care plans and treatment plans for a patient
   - get_patient_procedures: Retrieve medical procedures and interventions performed on a patient
   - get_patient_imaging_studies: Retrieve imaging studies and radiology reports for a patient
   - get_patient_immunizations: Retrieve vaccination history and immunization records for a patient
   - get_patient_allergies: Retrieve allergy information and adverse reactions for a patient


In [14]:
# Tool Function Implementations (Using generate_notes_for_type)
class ClinicalDataTools:
    """Clinical data retrieval tools for the OpenAI assistant"""
    
    def __init__(self):
        self.de_identifier = None  # Will be initialized later
        print("🔧 Clinical Data Tools initialized with all available note types")
    
    def get_patient_observations(self, patient_id: str) -> str:
        """
        Retrieve laboratory test results and clinical observations for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("observations", patient_id)
    
    def get_patient_conditions(self, patient_id: str) -> str:
        """
        Retrieve patient diagnosis information, medical conditions, and medical history
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("conditions", patient_id)
    
    def get_patient_medications(self, patient_id: str) -> str:
        """
        Retrieve current and past medications for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("medications", patient_id)

    def get_patient_careplans(self, patient_id: str) -> str:
        """
        Retrieve care plans and treatment plans for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("careplans", patient_id)

    def get_patient_procedures(self, patient_id: str) -> str:
        """
        Retrieve medical procedures and interventions performed on a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("procedures", patient_id)

    def get_patient_imaging_studies(self, patient_id: str) -> str:
        """
        Retrieve imaging studies and radiology reports for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("imaging_studies", patient_id)

    def get_patient_immunizations(self, patient_id: str) -> str:
        """
        Retrieve vaccination history and immunization records for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("immunizations", patient_id)

    def get_patient_allergies(self, patient_id: str) -> str:
        """
        Retrieve allergy information and adverse reactions for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("allergies", patient_id)

# Initialize the tools
clinical_tools = ClinicalDataTools()
print("✅ Clinical tools instance created with all available note types!")
print("📋 Available note types: observations, conditions, medications, careplans, procedures, imaging_studies, immunizations, allergies")

🔧 Clinical Data Tools initialized with all available note types
✅ Clinical tools instance created with all available note types!
📋 Available note types: observations, conditions, medications, careplans, procedures, imaging_studies, immunizations, allergies


In [15]:
# LLM Call and Prompt Management Functions
def build_system_prompt() -> str:
    """Build the system prompt for the clinical assistant"""
    return f"""You are an AI Clinical Reasoning Assistant with expertise in internal medicine. 
    Analyze clinical data and respond to patient-specific questions using structured reasoning.

    Available Tools:
    - get_patient_observations(): Laboratory test results and clinical observations
    - get_patient_conditions(): Diagnosis history, medical conditions, and medical history
    - get_patient_medications(): Current and past medications
    - get_patient_careplans(): Care plans and treatment plans
    - get_patient_procedures(): Medical procedures and interventions
    - get_patient_imaging_studies(): Imaging studies and radiology reports
    - get_patient_immunizations(): Vaccination history and immunization records
    - get_patient_allergies(): Allergy information and adverse reactions

    Clinical Reasoning Framework:
    1. Analyze the question to determine needed information
    2. Use appropriate tools to gather relevant patient data
    3. Synthesize findings from multiple data sources
    4. Provide clear, evidence-based responses with clinical reasoning

    Rules:
    - Use ONLY data provided by tool outputs
    - Reference relative timeframes when provided (e.g., "Day 0", "Post-op Day 3")
    - Acknowledge limitations if data is insufficient
    - Suggest additional information needed when applicable
    - Consider interactions between medications, conditions, and procedures
    - Always check for allergies before recommending treatments

    Current date: {datetime.now().strftime('%Y-%m-%d')}
    """

def build_messages(query: str, patient_id: str = None) -> List[Dict[str, str]]:
    """Build messages for the LLM call"""
    messages = [
        {"role": "system", "content": build_system_prompt()},
        {"role": "user", "content": f"Clinical query: {query}"}
    ]
    
    if patient_id:
        messages[-1]["content"] += f"\\nPatient ID: {patient_id}"
    
    return messages

def call_llm(client: OpenAI, config: ClinicalAssistantConfig, messages: List[Dict[str, str]]):
    """Make the actual LLM call"""
    return client.chat.completions.create(
        model=config.model,
        messages=messages,
        tools=TOOL_DEFINITIONS,
        tool_choice="auto",
        max_completion_tokens=config.max_completion_tokens,
        temperature=config.temperature
    )

In [16]:
# Clinical Assistant with Optional De-identification
class ClinicalAssistant:
    """Main clinical assistant that orchestrates OpenAI GPT-4o-mini model with tool calling"""
    
    def __init__(self, client: OpenAI, config: ClinicalAssistantConfig, tools: ClinicalDataTools):
        self.client = client
        self.config = config
        self.tools = tools
        
        # Map function names to actual methods
        self.function_map = {
            "get_patient_observations": self.tools.get_patient_observations,
            "get_patient_conditions": self.tools.get_patient_conditions,
            "get_patient_medications": self.tools.get_patient_medications,
            "get_patient_careplans": self.tools.get_patient_careplans,
            "get_patient_procedures": self.tools.get_patient_procedures,
            "get_patient_imaging_studies": self.tools.get_patient_imaging_studies,
            "get_patient_immunizations": self.tools.get_patient_immunizations,
            "get_patient_allergies": self.tools.get_patient_allergies
        }
    
    def de_identify_data(self, data: str) -> str:
        """De-identify text data using the existing deidentify function"""
        try:
            return deidentify(data)
        except Exception as e:
            print(f"Warning: De-identification failed: {e}")
            return data
    
    def execute_tool_call(self, function_name: str, arguments: Dict[str, Any], apply_deidentification: bool = False) -> str:
        """Execute a tool function call with optional de-identification"""
        if function_name not in self.function_map:
            return f"Error: Unknown function: {function_name}"
        print(f"Executing tool call: {function_name}")
        try:
            func = self.function_map[function_name]
            raw_result = func(**arguments)
            
            # Apply de-identification if requested
            if apply_deidentification and isinstance(raw_result, str):
                return self.de_identify_data(raw_result)
            
            return raw_result
            
        except Exception as e:
            return f"Error executing {function_name}: {str(e)}"
    
    def process_clinical_query(self, query: str, patient_id: str = None, apply_deidentification: bool = False) -> str:
        """
        Process a clinical query using OpenAI GPT-4o-mini with tool calling
        
        Args:
            query: The clinical question to answer
            patient_id: Optional patient ID if known
            apply_deidentification: Whether to apply de-identification to tool outputs
            
        Returns:
            Clinical assistant response
        """
        if not self.client:
            return "❌ OpenAI client not initialized. Please check your API key."
        
        try:
            # Get initial response from LLM
            messages = build_messages(query, patient_id)
            response = call_llm(self.client, self.config, messages)
            
            # Handle tool calls if any
            if response.choices[0].message.tool_calls:
                current_messages = messages.copy()
                current_messages.append({
                    "role": "assistant",
                    "content": response.choices[0].message.content,
                    "tool_calls": response.choices[0].message.tool_calls
                })
                
                # Process each tool call
                for tool_call in response.choices[0].message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    
                    # Execute tool call with optional de-identification
                    tool_result = self.execute_tool_call(function_name, function_args, apply_deidentification)
                    
                    current_messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": tool_result
                    })
                
                # Get final response with tool results
                final_response = call_llm(self.client, self.config, current_messages)
                return final_response.choices[0].message.content
            else:
                return response.choices[0].message.content
            
        except Exception as e:
            return f"❌ Error processing query: {str(e)}"

# Initialize the clinical assistant
if client:
    assistant = ClinicalAssistant(client, config, clinical_tools)
    print("✅ Clinical Assistant initialized with all 8 tool functions!")
else:
    assistant = None

✅ Clinical Assistant initialized with all 8 tool functions!


In [19]:
# Test Clinical Assistant with Integrated De-identification
print("🧪 Testing Clinical Assistant with Integrated De-identification")
print("=" * 60)

# Sample clinical queries to test the system with all available tools
sample_queries = [
    "What are this patient's current lab abnormalities and what do they suggest clinically?",
    "Can you summarize this patient's medical conditions and current medications?",
    "What procedures has this patient undergone and what was the care plan?",
    "Does this patient have any allergies I should be aware of before prescribing?",
    "What is this patient's vaccination status and immunization history?",
    "What imaging studies have been performed and what were the findings?",
    "Based on all available data, what is the comprehensive clinical picture?",
    "Are there any drug interactions or contraindications based on current medications and allergies?"
]

# Enhanced interactive testing function with de-identification options
def test_query(query_text: str, patient_id: str = "12345678", apply_deidentification: bool = True):
    """
    Test function for interactive querying with de-identification options
    
    Args:
        query_text: The clinical question to ask
        patient_id: Patient identifier (default: "12345678")
        apply_deidentification: Whether to apply de-identification (default: True)
    
    Returns:
        Assistant response
    """
    if not assistant:
        return "❌ Clinical Assistant not initialized. Please set your OpenAI API key."
    
    print(f"🔍 Patient ID: {patient_id}")
    print(f"📝 Query: {query_text}")
    print(f"🔒 De-identification: {'Enabled' if apply_deidentification else 'Disabled'}")
    print("\n🤖 Assistant Response:")
    print("-" * 60)
    
    response = assistant.process_clinical_query(query_text, patient_id, apply_deidentification)
    
    print("-" * 60)
    print(response)
    print("-" * 60)

🧪 Testing Clinical Assistant with Integrated De-identification


In [20]:


# Test with a sample patient ID (this will be de-identified)
test_patient_id = "e7a5d3dc-3484-a24e-6ef3-0737b403f950"
query = sample_queries[2]
test_query(query, test_patient_id, apply_deidentification=True)

🔍 Patient ID: e7a5d3dc-3484-a24e-6ef3-0737b403f950
📝 Query: What procedures has this patient undergone and what was the care plan?
🔒 De-identification: Enabled

🤖 Assistant Response:
------------------------------------------------------------
Executing tool call: get_patient_procedures

🔍 Starting de-identification process...
📏 Input size: 34292 characters (14873 tokens)
📊 Text exceeds safe input size of 5866 tokens. Splitting into chunks...
Executing tool call: get_patient_careplans

🔍 Starting de-identification process...
📏 Input size: 2937 characters (1097 tokens)
✅ Text fits in context window, processing as a single chunk.
✅ De-identification complete. Output size: 956 characters.
------------------------------------------------------------
Here is a summary of the procedures and care plans for patient ID e7a5d3dc-3484-a24e-6ef3-0737b403f950:

Procedures:
- The patient has undergone a wide range of procedures, including:
    - Measurement of respiratory function (for acute bronchi